<a href="https://colab.research.google.com/github/thofes/judging_SJ_BA/blob/main/Mark_11_yolo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Connect GDrive
from google.colab import drive
drive.mount('/content.gdrive')

#init Repository - YOLOv5
%cd /content


!git clone https://github.com/thofes/yolov5  # clone
%cd yolov5
%pip install -qr requirements.txt  # install

Mounted at /content.gdrive
/content
Cloning into 'yolov5'...
remote: Enumerating objects: 11166, done.
remote: Total 11166 (delta 0), reused 0 (delta 0), pack-reused 11166
Receiving objects: 100% (11166/11166), 11.18 MiB | 28.85 MiB/s, done.
Resolving deltas: 100% (7704/7704), done.
/content/yolov5
     |████████████████████████████████| 596 kB 33.6 MB/s 


Exception falls gar nichts entdeckt wir dim BIld

In [2]:
import os
from os.path import exists, join, basename, splitext
import math
from collections import OrderedDict
import json
import numpy as np
import cv2


PATH = "/content.gdrive/My Drive/Try"
label= []
try:
    label = next(os.walk(PATH))[1]
except StopIteration:
    pass
#data["labels"].append(label)
print(label)

for l in label:
  Path_jumpers = PATH + '/' + l
  print(Path_jumpers)
  Jumpers = []
  try:
    Jumpers = next(os.walk(Path_jumpers))[1]
  except StopIteration:
    pass
  print(Jumpers)

  for j in Jumpers:
    Path_Pic = Path_jumpers + '/' + j
    print(Path_Pic)

    %cd /content/yolov5
    !python detect.py --save-crop --source '{Path_Pic}' --project '{Path_Pic}' --save-txt


    with open(Path_Pic + '/exp/predictions.json', "r") as fp:
        data = json.load(fp)

    # convert lists to numpy arrays
    x_center = np.array(data["x_center"])
    y_center = np.array(data["y_center"])
    x1 = np.array(data["x1"])
    x2 = np.array(data["x2"])
    y1 = np.array(data["y1"])
    y2 = np.array(data["y2"])
    det = np.array(data["det"])
    width = 1920
    hight = 1080




    #detect movement mit check vor und zurück


    athlet_det_index = []
    y_centers = []

    for y in range(21):
      print("loop" , y)
      if det[y] != 0: #wird eventuellenötigt, falls keine detections gemacht wurden
        distance_next = []
        distance_index_next = []
        distance_previous = []
        distance_index_previous = []
        valid_dist_index_next = []
        valid_dist_index_previous = []
        valid_dist_index = []
        y_center_per_det = []
        y_center_abs_per_det = []

        if y == 0: #beim ersten Bild wird der Abstand der BB zum Folgebild berrechnet

            for x in range(det[y+1]):
                distance_index_previous.append(x) # wird aufgefüllt damit beim Vergleich nur die Indexe der distance_next Liste vorhanden bleiben            
                for i in range(det[y]):
                    distance_next.append(math.sqrt((x_center[sum(det[:y+1])+x]-x_center[sum(det[:y])+i])**2 + (y_center[sum(det[:y+1])+x]-y_center[sum(det[:y])+i])**2))
                    distance_index_next.append(i)
            

        if y > 0 and y < 20: #bei den mittleren Bildern wird der Abstand der BB zum Vorbild und Folgebild berrechnet
            
            for x in range(det[y+1]):
                
                for i in range(det[y]):
                    distance_next.append(math.sqrt((x_center[sum(det[:y+1])+x]-x_center[sum(det[:y])+i])**2 + (y_center[sum(det[:y+1])+x]-y_center[sum(det[:y])+i])**2))
                    distance_index_next.append(i)

            for x in range(det[y-1]):
                
                for i in range(det[y]):
                    distance_previous.append(math.sqrt((x_center[sum(det[:y-1])+x]-x_center[sum(det[:y])+i])**2 + (y_center[sum(det[:y-1])+x]-y_center[sum(det[:y])+i])**2))
                    distance_index_previous.append(i)
            #print(distance)
            
                  
        else: # Beim letzten Bild wird der Abstand zur BB des vorletzten Bildes berechnet
            for x in range(det[y-1]):
                
                for i in range(det[y]):
                    distance_previous.append(math.sqrt((x_center[sum(det[:y-1])+x]-x_center[sum(det[:y])+i])**2 + (y_center[sum(det[:y-1])+x]-y_center[sum(det[:y])+i])**2))
                    distance_index_previous.append(i)
        #print(x_center)          
        print("dist_next: ", distance_next)
        print("dist_prev: ", distance_previous)
            #print(distance_index)

        for x in range(det[y]): # y Koordinaten des Zentrums in Liste speichern
            y_center_per_det.append(y_center[sum(det[:y])+x])
            

        y_centers.append(y_center_per_det) # Erstellung einer Liste mit den Listen der einzelnen y Koordinaten der Detections in jedem Bild

        if y == 0: # beim ersten Bild nur vergleich mit dem nächsten
            for i, x in enumerate(distance_next): # Einschränkung der Detections anhand des Abstandes zum nächsten Bild
                if x > 15 and x < 70:
                    valid_dist_index.append(distance_index_next[i])

        if y > 0 and y < 20: # bei den mittleren Bildern werden beide Richtungen eingeschränkt
            for i, x in enumerate(distance_next): # Einschränkung der Detections anhand des Abstandes zum nächsten Bild
                if x > 15 and x < 70:
                    valid_dist_index_next.append(distance_index_next[i])
                    

            for i, x in enumerate(distance_previous): # Einschränkung der Detections anhand des Abstandes zum vorherigen Bild
                if x > 15 and x < 70:
                    valid_dist_index_previous.append(distance_index_previous[i])

            print("next: ",valid_dist_index_next)
            print("prev: ",valid_dist_index_previous)
            # Listeneinträge vergleichen und Übereinstimmungen zusammenführen

            #Problem wenn nur in eine Richtung detectiert wurde zb nur next hat eintrag previous nicht
            #s1 = set(valid_dist_index_next)
            #s2 = set(valid_dist_index_previous)
            #s3 = s1.intersection(s2)
            #valid_dist_index = list(s3)

            #wird einfach aufsumiert und dann geordnet
            valid_dist_index = valid_dist_index_next + valid_dist_index_previous
            

        if y == 20: # beim letzten Bid wieder nur vergleich mit dem vorherigen Bild
            for i, x in enumerate(distance_previous): # Einschränkung der Detections anhand des Abstandes zum vorherigen Bild
                if x > 15 and x < 70:
                    valid_dist_index.append(distance_index_previous[i])
        




        print("validdetList: ", valid_dist_index)

        if len(valid_dist_index) > 1: # Falls für eine Bildbox mehrere mögliche Verbindungen zum Folgebild gefunden wurden, werden die Duplikate entfernt
            valid_dist_index = list(OrderedDict.fromkeys(valid_dist_index))
            print("nach order: ", valid_dist_index)

        if len(valid_dist_index) == 1: # wenn nur eine vorhanden ist wird diese verwendet
            athlet_det_index.append(valid_dist_index[0])

        if y == 0: # Für das erste Bild
            
            if len(valid_dist_index) > 1: # falls mehrere möglich sind, wird die BB mit der zentralste Y Koordinate gewählt
                for dist in valid_dist_index:
                    y_center_abs_per_det.append(abs(y_center[sum(det[:y])+dist]-hight/2))
                closest = min(y_center_abs_per_det)
                min_index = y_center_abs_per_det.index(closest)            
                athlet_det_index.append(valid_dist_index[min_index]) # index der Detection vom ersten Bild gespeichert
            
            elif len(valid_dist_index) == 0: # falls keine Movements aus den BB erkannt werden - auch zetralste Y-Koordinate
                for x in range(det[y]):
                    y_center_abs_per_det.append(abs(y_center[sum(det[:y])+x]-hight/2)) 
                closest = min(y_center_abs_per_det)
                min_index = y_center_abs_per_det.index(closest)
                athlet_det_index.append(min_index) # index der Detection vom ersten Bild gespeichert


        elif y < 21: # Für alle weiteren Bilder 

            if len(valid_dist_index) > 1: # falls mehrere möglich sind, wird jene BB mit dem geringsten Abstand zur Y-Koordinate der vorher geählten BB ausgewählt
                dist_to_previous_y_center = []

                for i, x in enumerate(valid_dist_index):
                    dist_to_previous_y_center.append(abs(y_center_per_det[x]-y_center[sum(det[:y-1])+athlet_det_index[y-1]]))
                closest = min(dist_to_previous_y_center)
                min_index = dist_to_previous_y_center.index(closest)
                athlet_det_index.append(valid_dist_index[min_index]) # index der Detection wird gespeichert           

                #optional einfach die zentralste Detection laut Y-Kooridnate
                #for x in range(det[y]):
                    #y_center_abs_per_det.append(abs(y_center[sum(det[:y])+x]-hight/2)) 
                #closest = min(y_center_abs_per_det)
                #min_index = y_center_abs_per_det.index(closest)
                #athlet_det_index.append(valid_dist_index[min_index]) # index der Detection vom Bild wird gespeichert

            if len(valid_dist_index) == 0: #falls keine Movements aus den BB erkannt werden - auch zetralste Y-Koordinate oder vergleichen mit dem vorherigen Bild
                print("BP")

        print("athlet_det:" , athlet_det_index)
      else:
        athlet_det_index.append(0) # so wird einfach die erste detection dann beim ersten Bild zugeschnitten

    print(athlet_det_index)










    #crop images
    for i,x in enumerate(athlet_det_index):
        if x != 'X':
            #print(sum(det[:i]))
            #print(x)
            x_1 = x1[sum(det[:i])+x]-50
            x_2 = x2[sum(det[:i])+x]+50
            y_1 = y1[sum(det[:i])+x]-50
            y_2 = y2[sum(det[:i])+x]+50

            if x_1 < 0:
                x_1 = 0

            if x_2 > width:
                x_2 = width

            if y_1 < 0:
                y_1 = 0

            if y_2 > hight:
                y_2 = hight


            path = Path_Pic + '/' + str(i+1) + '.jpg'
            print(path)
            im = cv2.imread(path)
            #print(x_1, x_2, y_1, y_2)
            crop = im[int(y_1):int(y_2), int(x_1):int(x_2)] 
            #save_dir.parent.mkdir(parents=True, exist_ok=True)  # make directory
            save_path = Path_Pic + '/exp/crop' + str(i+1) + '.jpg'
            cv2.imwrite(save_path, crop) # speicher nur den zugeschnittenen Springer







['Telemark', 'NoTelemark']
/content.gdrive/My Drive/Try/Telemark
['1138', '1402', '1572', '1607', '1626', '1625', '1837', '1840', '1841', '2012', '2009', '2010', '901', '914', '913', '1038']
/content.gdrive/My Drive/Try/Telemark/1138
/content/yolov5
YOLOv5 🚀 v6.1-33-g7c874ca torch 1.10.0+cu111 CPU

100% 14.1M/14.1M [00:00<00:00, 244MB/s]

Fusing layers... 
Model Summary: 213 layers, 7225885 parameters, 0 gradients, 16.5 GFLOPs
BP  /content.gdrive/My Drive/Try/Telemark/1138
images:  ['/content.gdrive/My Drive/Try/Telemark/1138/1.jpg', '/content.gdrive/My Drive/Try/Telemark/1138/2.jpg', '/content.gdrive/My Drive/Try/Telemark/1138/3.jpg', '/content.gdrive/My Drive/Try/Telemark/1138/4.jpg', '/content.gdrive/My Drive/Try/Telemark/1138/5.jpg', '/content.gdrive/My Drive/Try/Telemark/1138/6.jpg', '/content.gdrive/My Drive/Try/Telemark/1138/7.jpg', '/content.gdrive/My Drive/Try/Telemark/1138/8.jpg', '/content.gdrive/My Drive/Try/Telemark/1138/9.jpg', '/content.gdrive/My Drive/Try/Telemark/1138/

IndexError: ignored

In [26]:
#without loop

import os
from os.path import exists, join, basename, splitext
import math
from collections import OrderedDict
import json
import numpy as np
import cv2


Path_Pic = "/content.gdrive/My Drive/Try/Telemark/1138"


#%cd /content/yolov5
#!python detect.py --save-crop --source '{Path_Pic}' --project '{Path_Pic}' --save-txt


with open(Path_Pic + '/exp/predictions.json', "r") as fp:
    data = json.load(fp)

# convert lists to numpy arrays
x_center = np.array(data["x_center"])
y_center = np.array(data["y_center"])
x1 = np.array(data["x1"])
x2 = np.array(data["x2"])
y1 = np.array(data["y1"])
y2 = np.array(data["y2"])
det = np.array(data["det"])
width = 1920
hight = 1080

print(x_center)
print(y_center)
print(det)




#detect movement mit check vor und zurück


athlet_det_index = []
y_centers = []

for y in range(21):
  print("loop" , y)
  if det[y] != 0: #wird eventuellenötigt, falls keine detections gemacht wurden
    distance_next = []
    distance_index_next = []
    distance_previous = []
    distance_index_previous = []
    valid_dist_index_next = []
    valid_dist_index_previous = []
    valid_dist_index = []
    y_center_per_det = []
    y_center_abs_per_det = []

    if y == 0: #beim ersten Bild wird der Abstand der BB zum Folgebild berrechnet

        for x in range(det[y+1]):
            distance_index_previous.append(x) # wird aufgefüllt damit beim Vergleich nur die Indexe der distance_next Liste vorhanden bleiben            
            for i in range(det[y]):
                distance_next.append(math.sqrt((x_center[sum(det[:y+1])+x]-x_center[sum(det[:y])+i])**2 + (y_center[sum(det[:y+1])+x]-y_center[sum(det[:y])+i])**2))
                distance_index_next.append(i)
        

    if y > 0 and y < 20: #bei den mittleren Bildern wird der Abstand der BB zum Vorbild und Folgebild berrechnet
        
        for x in range(det[y+1]):
            
            for i in range(det[y]):
                distance_next.append(math.sqrt((x_center[sum(det[:y+1])+x]-x_center[sum(det[:y])+i])**2 + (y_center[sum(det[:y+1])+x]-y_center[sum(det[:y])+i])**2))
                distance_index_next.append(i)

        for x in range(det[y-1]):
            
            for i in range(det[y]):
                distance_previous.append(math.sqrt((x_center[sum(det[:y-1])+x]-x_center[sum(det[:y])+i])**2 + (y_center[sum(det[:y-1])+x]-y_center[sum(det[:y])+i])**2))
                distance_index_previous.append(i)
        #print(distance)
        
              
    else: # Beim letzten Bild wird der Abstand zur BB des vorletzten Bildes berechnet
        for x in range(det[y-1]):
            
            for i in range(det[y]):
                distance_previous.append(math.sqrt((x_center[sum(det[:y-1])+x]-x_center[sum(det[:y])+i])**2 + (y_center[sum(det[:y-1])+x]-y_center[sum(det[:y])+i])**2))
                distance_index_previous.append(i)
    #print(x_center)          
    print("dist_next: ", distance_next)
    print("dist_prev: ", distance_previous)
        #print(distance_index)

    for x in range(det[y]): # y Koordinaten des Zentrums in Liste speichern
        y_center_per_det.append(y_center[sum(det[:y])+x])
        

    y_centers.append(y_center_per_det) # Erstellung einer Liste mit den Listen der einzelnen y Koordinaten der Detections in jedem Bild

    if y == 0: # beim ersten Bild nur vergleich mit dem nächsten
        for i, x in enumerate(distance_next): # Einschränkung der Detections anhand des Abstandes zum nächsten Bild
            
            if x > 20 and x < 75:
                valid_dist_index.append(distance_index_next[i])
                print("IF: ", x)
                #print(i%det[y])
                #print("test: ", abs(y_center[sum(det[:y])+i%det[y]]-y_center[sum(det[:y-1])+i%det[y-1]]))

    if y > 0 and y < 20: # bei den mittleren Bildern werden beide Richtungen eingeschränkt
        for i, x in enumerate(distance_next): # Einschränkung der Detections anhand des Abstandes zum nächsten Bild
            if x > 20 and x < 75:
                valid_dist_index_next.append(distance_index_next[i])
                print("IF: ", x)
                print(i%det[y])
                print("test: ", abs(y_center[sum(det[:y])+i%det[y]]-y_center[sum(det[:y-1])+athlet_det_index[y-1]]))
                

        for i, x in enumerate(distance_previous): # Einschränkung der Detections anhand des Abstandes zum vorherigen Bild
            
            if x > 20 and x < 75:
                valid_dist_index_previous.append(distance_index_previous[i])
                print("IF: ", x)
                print(i%det[y])
                print("test: ", abs(y_center[sum(det[:y-1])+i%det[y-1]]-y_center[sum(det[:y])++athlet_det_index[y-1]]))

        print("next: ",valid_dist_index_next)
        print("prev: ",valid_dist_index_previous)
        # Listeneinträge vergleichen und Übereinstimmungen zusammenführen

        #Problem wenn nur in eine Richtung detectiert wurde zb nur next hat eintrag previous nicht
        #s1 = set(valid_dist_index_next)
        #s2 = set(valid_dist_index_previous)
        #s3 = s1.intersection(s2)
        #valid_dist_index = list(s3)

        #wird einfach aufsumiert und dann geordnet
        valid_dist_index = valid_dist_index_next + valid_dist_index_previous
        

    if y == 20: # beim letzten Bid wieder nur vergleich mit dem vorherigen Bild
        for i, x in enumerate(distance_previous): # Einschränkung der Detections anhand des Abstandes zum vorherigen Bild
            
            if x > 20 and x < 75:
                valid_dist_index.append(distance_index_previous[i])
                print("IF: ", x)
                print(i%det[y])
                print("test: ", abs(y_center[sum(det[:y-1])+i%det[y-1]]-y_center[sum(det[:y])++athlet_det_index[y-1]]))
    




    print("validdetList: ", valid_dist_index)

    if len(valid_dist_index) > 1: # Falls für eine Bildbox mehrere mögliche Verbindungen zum Folgebild gefunden wurden, werden die Duplikate entfernt
        valid_dist_index = list(OrderedDict.fromkeys(valid_dist_index))
        print("nach order: ", valid_dist_index)

    if len(valid_dist_index) == 1: # wenn nur eine vorhanden ist wird diese verwendet
        athlet_det_index.append(valid_dist_index[0])

    if y == 0: # Für das erste Bild
        
        if len(valid_dist_index) > 1: # falls mehrere möglich sind, wird die BB mit der zentralste Y Koordinate gewählt
            for dist in valid_dist_index:
                y_center_abs_per_det.append(abs(y_center[sum(det[:y])+dist]-hight/2))
            closest = min(y_center_abs_per_det)
            min_index = y_center_abs_per_det.index(closest)            
            athlet_det_index.append(valid_dist_index[min_index]) # index der Detection vom ersten Bild gespeichert
        
        elif len(valid_dist_index) == 0: # falls keine Movements aus den BB erkannt werden - auch zetralste Y-Koordinate
            for x in range(det[y]):
                y_center_abs_per_det.append(abs(y_center[sum(det[:y])+x]-hight/2)) 
            closest = min(y_center_abs_per_det)
            min_index = y_center_abs_per_det.index(closest)
            athlet_det_index.append(min_index) # index der Detection vom ersten Bild gespeichert


    elif y < 21: # Für alle weiteren Bilder 

        if len(valid_dist_index) > 1: # falls mehrere möglich sind, wird jene BB mit dem geringsten Abstand zur Y-Koordinate der vorher geählten BB ausgewählt
            dist_to_previous_y_center = []

            for i, x in enumerate(valid_dist_index):
                dist_to_previous_y_center.append(abs(y_center_per_det[x]-y_center[sum(det[:y-1])+athlet_det_index[y-1]]))
            closest = min(dist_to_previous_y_center)
            min_index = dist_to_previous_y_center.index(closest)
            athlet_det_index.append(valid_dist_index[min_index]) # index der Detection wird gespeichert           

            #optional einfach die zentralste Detection laut Y-Kooridnate
            #for x in range(det[y]):
                #y_center_abs_per_det.append(abs(y_center[sum(det[:y])+x]-hight/2)) 
            #closest = min(y_center_abs_per_det)
            #min_index = y_center_abs_per_det.index(closest)
            #athlet_det_index.append(valid_dist_index[min_index]) # index der Detection vom Bild wird gespeichert

        if len(valid_dist_index) == 0: #falls keine Movements aus den BB erkannt werden - auch zetralste Y-Koordinate oder vergleichen mit dem vorherigen Bild
            athlet_det_index.append(999)

    print("athlet_det:" , athlet_det_index)

  else:
    athlet_det_index.append(999) # so wird einfach die erste detection dann beim ersten Bild zugeschnitten
print(athlet_det_index)










#crop images
for i,x in enumerate(athlet_det_index):
    if x != 999:
        #print(sum(det[:i]))
        print(sum(det[:i])+x)
        print(i)
        print(x)
        x_1 = x1[sum(det[:i])+x]-50
        x_2 = x2[sum(det[:i])+x]+50
        y_1 = y1[sum(det[:i])+x]-50
        y_2 = y2[sum(det[:i])+x]+50

        if x_1 < 0:
            x_1 = 0

        if x_2 > width:
            x_2 = width

        if y_1 < 0:
            y_1 = 0

        if y_2 > hight:
            y_2 = hight

    else:
        x_1 = 0
        x_2 = 1
        y_1 = 0
        y_2 = 1


    path = Path_Pic + '/' + str(i+1) + '.jpg'
    print(path)
    im = cv2.imread(path)
    #print(x_1, x_2, y_1, y_2)
    crop = im[int(y_1):int(y_2), int(x_1):int(x_2)] 
    #save_dir.parent.mkdir(parents=True, exist_ok=True)  # make directory
    save_path = Path_Pic + '/exp/crop' + str(i+1) + '.jpg'
    cv2.imwrite(save_path, crop) # speicher nur den zugeschnittenen Springer

[1814.00001526 1331.50005341  381.49998665  850.50001144 1238.99997711
  853.50002289 1814.50000763 1331.50005341  381.49998665  848.99997711
 1285.99994659  850.50001144 1332.00004578 1815.          849.5000267
  380.00000954 1332.50003815 1815.         1330.99994659  383.50001335
  851.49999619 1382.99995422  849.5000267  1329.99996185 1815.
  379.49998856  848.99997711 1425.99998474 1815.49999237 1331.50005341
  382.5         851.49999619 1474.00005341  851.00000381 1331.50005341
 1814.50000763  378.99999619  849.5000267  1517.49996185  850.50001144
 1330.99994659 1815.49999237  380.00000954  849.5000267  1572.00004578
 1326.50001526  851.00000381 1330.49995422 1815.          380.00000954
  848.99997711 1615.99994659  850.50001144 1330.99994659 1815.49999237
  378.99999619  848.99997711 1665.         1507.99999237  851.00000381
 1330.49995422  378.50000381 1815.49999237  849.5000267  1714.50004578
  851.00000381 1331.50005341  849.5000267   378.99999619 1815.99998474
 1762.99999237 